In [1]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn as sk
import re
#imputer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

In [2]:
# Read in the data
books = pd.read_csv('BX-Books.csv')
ratings = pd.read_csv('BX-Ratings.csv')
users = pd.read_csv('BX-Users.csv')

# Merge ratings with books
merged_data = pd.merge(ratings, books, on='ISBN', how='inner')

# Merge merged_data with users 
merged_data = pd.merge(merged_data, users, on='User-ID', how='inner')

#save merged data
merged_data.to_csv('merged_data.csv')

In [3]:
def preprocess_title(title):
    # Convert to lowercase and remove leading articles
    title = title.lower()
    title = re.sub(r'^\b(the|a|an)\s+', '', title)
    return title

In [4]:
# Load your dataset
data = pd.read_csv('merged_data.csv')

# Preprocess the 'Book-Title' column
data['Processed-Title'] = data['Book-Title'].apply(preprocess_title)

In [5]:
# Load your series names (assuming you have them in a CSV with 'Simplified-Title' column)
popular_titles = pd.read_csv('popular_titles.csv')
popular_titles['Simplified-Title'] = popular_titles['Simplified-Title'].apply(preprocess_title)
popular_titles['full-title'] = popular_titles['titles']

# Map each title to the series it belongs to, if any
def map_to_series(title, series_list):
    for series in series_list:
        if series in title:
            return series
    return title  # If no series match, return the title as is

# Apply the mapping
series_list = popular_titles['Simplified-Title'].unique()
data['Series-Title'] = data['Processed-Title'].apply(lambda title: map_to_series(title, series_list))

# Aggregate the reviews by series/title
aggregated_data = data.groupby('Series-Title').agg({'Book-Rating': 'sum', 'User-ID': 'count'}).rename(columns={'User-ID': 'Review-Count'})
aggregated_data = aggregated_data.sort_values(by='Review-Count', ascending=False)

# Select the top 100
top_100_series = aggregated_data.head(100)


In [6]:
# Save to CSV
top_100_series.to_csv('top_100_series.csv')

In [7]:
#get top_100_series to a pandas dataframe
top_100_series = pd.read_csv('top_100_series.csv')
#I want the index to be considerd as a column
top_100_series.reset_index(inplace=True)
#delete the index column
top_100_series.drop(columns=['index'], inplace=True)
#I want top_100_series["Series-Title"] to be printed in an array
series_titles = top_100_series["Series-Title"].to_numpy()
print(series_titles)

['ring' 'harry potter' 'bone' 'stephanie plum' 'left behind'
 'bridget jones' 'vampire chronicles' 'wild animus' 'da vinci code'
 'divine secrets of the ya-ya sisterhood: a novel' 'secret life of bees'
 'nanny diaries: a novel' 'red tent (bestselling backlist)'
 'painted house' 'his dark materials' "earth's children" 'hobbit'
 'alex cross' 'life of pi' 'dark tower' 'wicked' 'discworld'
 'angels & demons' 'summons'
 "where the heart is (oprah's book club (paperback))" 'notebook'
 'catcher in the rye' 'dune' "no. 1 ladies' detective agency"
 'snow falling on cedars' "pilot's wife : a novel" 'wheel of time'
 'timeline' 'testament' "girls' guide to hunting and fishing"
 'to kill a mockingbird' 'summer sisters'
 "tuesdays with morrie: an old man, a young man, and life's greatest lesson"
 'five people you meet in heaven' 'good in bed' 'house of sand and fog'
 '1st to die: a novel' 'kay scarpetta' 'interview with the vampire'
 "suzanne's diary for nicholas" 'mitford years' 'joy luck club'
 'b

In [8]:
#based on this mapping, map the series titles to their genres adding a new column to the top_100_series dataframe
book_genre_map = {
    'ring': 'ring - Fantasy', 
    'harry potter': 'harry potter - Fantasy', 
    'bone': 'bone - Graphic Novel', 
    'stephanie plum': 'stephanie plum - Mystery', 
    'left behind': 'left behind - Christian Fiction',
    'bridget jones': 'bridget jones - Romance', 
    'vampire chronicles': 'vampire chronicles - Horror', 
    'wild animus': 'wild animus - Fiction', 
    'da vinci code': 'da vinci code - Thriller', 
    'divine secrets of the ya-ya sisterhood: a novel': 'divine secrets of the ya-ya sisterhood: a novel - Southern Fiction',
    'secret life of bees': 'secret life of bees - Historical Fiction', 
    'nanny diaries: a novel': 'nanny diaries: a novel - Fiction', 
    'red tent (bestselling backlist)': 'red tent (bestselling backlist) - Historical Fiction', 
    'painted house': 'painted house - Fiction', 
    'his dark materials': 'his dark materials - Fantasy', 
    "earth's children": "earth's children - Historical Fiction", 
    'hobbit': 'hobbit - Fantasy', 
    'alex cross': 'alex cross - Thriller', 
    'life of pi': 'life of pi - Adventure Fiction', 
    'dark tower': 'dark tower - Fantasy', 
    'wicked': 'wicked - Fantasy', 
    'discworld': 'discworld - Fantasy', 
    'angels & demons': 'angels & demons - Thriller', 
    'summons': 'summons - Thriller', 
    "where the heart is (oprah's book club (paperback))": "where the heart is (oprah's book club (paperback)) - Fiction", 
    'notebook': 'notebook - Romance', 
    'catcher in the rye': 'catcher in the rye - Classic', 
    'dune': 'dune - Science Fiction', 
    "no. 1 ladies' detective agency": "no. 1 ladies' detective agency - Mystery", 
    'snow falling on cedars': 'snow falling on cedars - Historical Fiction', 
    "pilot's wife : a novel": "pilot's wife : a novel - Fiction", 
    'wheel of time': 'wheel of time - Fantasy', 
    'timeline': 'timeline - Science Fiction', 
    'testament': 'testament - Thriller', 
    "girls' guide to hunting and fishing": "girls' guide to hunting and fishing - Chick Lit", 
    'to kill a mockingbird': 'to kill a mockingbird - Classic', 
    'summer sisters': 'summer sisters - Fiction', 
    "tuesdays with morrie: an old man, a young man, and life's greatest lesson": "tuesdays with morrie: an old man, a young man, and life's greatest lesson - Non-fiction", 
    'five people you meet in heaven': 'five people you meet in heaven - Fiction', 
    'good in bed': 'good in bed - Chick Lit', 
    'house of sand and fog': 'house of sand and fog - Fiction', 
    '1st to die: a novel': '1st to die: a novel - Thriller', 
    'kay scarpetta': 'kay scarpetta - Mystery', 
    'interview with the vampire': 'interview with the vampire - Horror', 
    "suzanne's diary for nicholas": "suzanne's diary for nicholas - Romance", 
    'mitford years': 'mitford years - Christian Fiction', 
    'joy luck club': 'joy luck club - Fiction', 
    'beach house': 'beach house - Fiction', 
    'reader': 'reader - Historical Fiction', 
    'horse whisperer': 'horse whisperer - Romance', 
    'firm': 'firm - Thriller', 
    'poisonwood bible: a novel': 'poisonwood bible: a novel - Historical Fiction', 
    'me talk pretty one day': 'me talk pretty one day - Humor', 
    'pelican brief': 'pelican brief - Thriller', 
    "she's come undone (oprah's book club)": "she's come undone (oprah's book club) - Fiction", 
    'brethren': 'brethren - Thriller', 
    'time to kill': 'time to kill - Thriller', 
    'chamber': 'chamber - Thriller', 
    "she's come undone (oprah's book club (paperback))": "she's come undone (oprah's book club (paperback)) - Fiction", 
    'dreamcatcher': 'dreamcatcher - Horror', 
    'jurassic park': 'jurassic park - Science Fiction', 
    'partner': 'partner - Thriller', 
    'street lawyer': 'street lawyer - Legal Thriller', 
    'shipping news': 'shipping news - Fiction', 
    'fahrenheit 451': 'fahrenheit 451 - Dystopian', 
    'bean trees': 'bean trees - Fiction', 
    'runaway jury': 'runaway jury - Legal Thriller', 
    'client': 'client - Legal Thriller', 
    'sword of truth': 'sword of truth - Fantasy', 
    'rainmaker': 'rainmaker - Legal Thriller', 
    'little altars everywhere: a novel': 'little altars everywhere: a novel - Fiction', 
    'empire falls': 'empire falls - Fiction', 
    'prayer for owen meany': 'prayer for owen meany - Fiction', 
    'king of torts': 'king of torts - Legal Thriller', 
    'artemis fowl': 'artemis fowl - Fantasy', 
    'bridges of madison county': 'bridges of madison county - Romance', 
    'white oleander : a novel': 'white oleander : a novel - Fiction', 
    'nickel and dimed: on (not) getting by in america': 'nickel and dimed: on (not) getting by in america - Non-fiction', 
    'message in a bottle': 'message in a bottle - Romance', 
    'wrinkle in time': 'wrinkle in time - Science Fiction', 
    'red dragon': 'red dragon - Thriller', 
    "tom clancy's op center": "tom clancy's op center - Thriller", 
    'hannibal': 'hannibal - Thriller', 
    'color purple': 'color purple - Historical Fiction', 
    "white oleander : a novel (oprah's book club)": "white oleander : a novel (oprah's book club) - Fiction", 
    'walk to remember': 'walk to remember - Romance', 
    'skipping christmas': 'skipping christmas - Humor', 
    'confessions of a shopaholic (summer display opportunity)': 'confessions of a shopaholic (summer display opportunity) - Chick Lit', 
    'stupid white men ...and other sorry excuses for the state of the nation!': 'stupid white men ...and other sorry excuses for the state of the nation! - Political Satire', 
    'while i was gone': 'while i was gone - Fiction', 
    'redwall': 'redwall - Fantasy', 
    'fast food nation: the dark side of the all-american meal': 'fast food nation: the dark side of the all-american meal - Non-fiction', 
    'bel canto: a novel': 'bel canto: a novel - Fiction', 
    "ender's game (ender wiggins saga (paperback))": "ender's game (ender wiggins saga (paperback)) - Science Fiction", 
    'bend in the road': 'bend in the road - Romance', 
    'midwives: a novel': 'midwives: a novel - Fiction', 
    "hitchhiker's guide to the galaxy": "hitchhiker's guide to the galaxy - Science Fiction", 
    'american gods': 'american gods - Fantasy', 
    'how to be good': 'how to be good - Fiction', 
    'into thin air : a personal account of the mt. everest disaster': 'into thin air : a personal account of the mt. everest disaster - Non-fiction'
}
top_100_series.to_csv('top_100_series.csv')

In [9]:
#in Series-Title column, filter out thoses reviews that are not in the top_100_series dataframe
data = data[data['Series-Title'].isin(top_100_series['Series-Title'])]

#add the genre column to the data dataframe based on top_100_series dataframe
data['Genre'] = data['Series-Title'].map(book_genre_map)
data['Genre'] = data['Genre'].str.split(' - ').str[1]
data['Genre'] = data['Genre'].replace('Christian Fiction', 'Fiction')
data['Genre'] = data['Genre'].replace('Southern Fiction', 'Fiction')
data['Genre'] = data['Genre'].replace('Adventure Fiction', 'Fiction')
data = data[data['Genre'] != 'Chick Lit']
data = data[data['Genre'] != 'Classic']
data = data[data['Genre'] != 'Humor']
data = data[data['Genre'] != 'Graphic Novel']
data = data[data['Genre'] != 'Dystopian']
data = data[data['Genre'] != 'Political Satire']
data = data[data['Genre'] != 'Non-fiction']

data.head(10)
#summarize the data
data.describe()
data.to_csv('data.csv')
